In [39]:
import csv
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from lxml import html
try:
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen
import dateutil
from dateutil.parser import *
from datetime import datetime
import pathlib
from htmldate import find_date
import trafilatura


In [37]:
# process statewise GERMAN VERSION 



def process_state_DE(state):

    agile_regex = re.compile(r'\bagility\b|\bagil\w{0,2}\b|\bagilität\b', re.IGNORECASE | re.UNICODE)

    agile_method_regex = re.compile(r'\bagil\w{0,2}\b((?![.]|agil\w{0,2}).)*?\bmethod\w*?\b', re.IGNORECASE | re.UNICODE)

    agile_context_regex = re.compile(r'\s*([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+agil.*?\s+([^\s]+)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+', re.IGNORECASE | re.UNICODE)

    csv_main = csv.DictReader(open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/CSVs/Germany/agile_sites_output_Germany_{}.csv".format(state)), fieldnames=["id", 'url', 'domain', 'date1', 'date2', 'date3', 'heading'])
    array = {}
    for line in csv_main: 
        array[line["id"]] = line
    text_dir = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/TextFiles/Germany/{}".format(state)
    pathlib.Path(text_dir).mkdir(parents=True, exist_ok=True)



######################## STILL TO BE ADAPTED FROM THE UK ######################





    for id, line in array.items():
        print("processing {}".format(id))
        id = line["id"]
        domain = line["domain"]
        path = '/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/HTMLs/Germany/{}/{}/{}.html'.format(state, domain, id)
        try:  
            soup = BeautifulSoup(open(path), "html.parser")
        except UnicodeDecodeError:
            soup = BeautifulSoup(open(path, encoding='windows-1252'), "html.parser")    
                
        #deleting all the Script and Style Elements
        for script in soup(["script", "style", "a"]):
            script.decompose()    # rip it out
                
         # get text
        doctext = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in doctext.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        doctext = '\n'.join(chunk for chunk in chunks if chunk)
        
        #old 'simple' cleaning         
        #doctext = soup.get_text().replace("\n", " ")
        with open("{}/{}.txt".format(text_dir, id), "w") as textfile:
            textfile.write(doctext)
        
        #Getting date4
        date4_element = soup.select_one("span.date")
        date4 = ""
        if date4_element is not None:
            date4 = date4_element.get_text()
        
        #Getting date5

        htmlparser = etree.HTMLParser()    
        try:
            tree = etree.parse(open(path), htmlparser)
        except UnicodeDecodeError:
            tree = etree.parse(open(path, encoding='windows-1252'), htmlparser)
        date5 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 23)")

        # a special cases for date 5
        date5_1 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 25)")


        # Getting date6

        date6_element = soup.select_one("h1#page-title + p")
        date6 = ""
        if date6_element is not None:
            date6 = date6_element.get_text()


        # Getting date7 from National Archives --> the date the site was archived on --> the date the site was released on would have been even earlier

        date7 = tree.xpath("substring(substring-after(/html/head/script/text(), 'timestamp'),9,14)")

        #Getting date with the htmldate package
        htmldate = find_date(tree)


        # assinging the already crawled dates
        date1 = line["date1"]
        date2 = line["date2"]


        # Storing the oldest date as final date variable in python date format
        date_vars = [date1, date2, date4, date5, date5_1, date6, date7]
        
        final_date = None 

        for date_var in date_vars:
            try:
                if final_date is None:
                    final_date = parse(date_var, ignoretz = True)
                elif parse(date_var, ignoretz=True) < final_date:
                    final_date = parse(date_var, ignoretz=True)
            except dateutil.parser._parser.ParserError:
                pass 


        #finding the matches for agile and agility
        agile_term = []
        agile_term = agile_regex.findall(doctext)

        #removing white space, dot etc. from the end of the term
        
        signs=[' ', '.',':',';']
        
       # print(agile_term_match)
        agile_term=[]
        
        if agile_term_match != []:
            for item in agile_term_match:
                if item[-1] in signs:
                   # print("item", item)
                    newitem = item[:-1]
                   # print("newitem", newitem)
                else:
                    newitem = item
                newitem = newitem.lower()
                agile_term.append(newitem)

        #print("newitem", newitem)
        #print("agile_term",agile_term)

        #finding the matches for agile...methods
        agile_method = []
        agile_method = agile_method_regex.findall(doctext.lower())


        # finding the context for the agile
        agile_context = []
        agile_context = agile_context_regex.search(doctext)

        agile_context_pre = ""
        agile_context_post = ""
        
        if agile_context is not None:
            agile_context_pre = " ".join(agile_context.group(1,2,3,4))
            agile_context_post = " ".join(agile_context.group(5,6,7,8))
        

        # assigning all the variables to the items

        if len(agile_term) == 0:
            line["agile_term"] = "" 
        else:
            line["agile_term"] = ",".join(agile_term)
        
        if len(agile_method) == 0:
            line["agile_method"] = "" 
        else:
            line["agile_method"] = ",".join(agile_method)

        line["agile_context_pre"] = agile_context_pre
        line["agile_context_post"] = agile_context_post 
        line["date4"] = date4
        line["date5"] = date5
        line["date5_1"] = date5_1
        line["date6"] = date6
        line["date7"] = date7
        line["final_date"] = final_date
        line["htmldate"] = htmldate
        line["country"] = "Germany"
        line["level"] = state
        line["text_file_loc"] = "{}/{}.txt".format(text_dir, id)
    
    outputfile = open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/CSVs/Germany/{}.csv".format(state), "w")
    writer = csv.DictWriter(outputfile, fieldnames=["id", "country", "level", 'url', 'domain', 'date1', 'date2', 'date3', 'date4', 'date5', 'date5_1', 'date6', 'date7', 'final_date', 'htmldate', 'heading', 'agile_term', 'agile_method', 'agile_context_pre', 'agile_context_post', 'text_file_loc'])
    writer.writeheader()
    for id, line in array.items():
        writer.writerow(line)
    outputfile.close()


In [44]:
# process statewise UK VERSION --> type "General"


def process_state_UK(state):

    agile_regex = re.compile(r'\bagility\b|\bagile\b', re.IGNORECASE | re.UNICODE)

    agile_method_regex = re.compile(r'\bagile\b(?:(?![.]|agile).)*?\bmethod\w*?\b', re.IGNORECASE | re.UNICODE)

    agile_context_regex = re.compile(r'\s*([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+agil.*?\s+([^\s]+)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+', re.IGNORECASE | re.UNICODE)

    csv_main = csv.DictReader(open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/CSVs/UK/SAVEPLACE/agile_sites_output_UK_{}.csv".format(state)), fieldnames=["id", 'url', 'domain', 'date1', 'date2', 'date3', 'heading'])
    array = {}
    for line in csv_main: 
        array[line["id"]] = line
    text_dir = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/TextFiles/UK/{}".format(state)
    pathlib.Path(text_dir).mkdir(parents=True, exist_ok=True)

    for id, line in array.items():
        print("processing {}".format(id))
        id = line["id"]
        domain = line["domain"]
        url = line["url"]
        path = '/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/HTMLs/UK/SAVEPLACE/{}/{}/{}.html'.format(state, domain, id)
        try:  
            soup = BeautifulSoup(open(path), "html.parser")
        except UnicodeDecodeError:
            soup = BeautifulSoup(open(path, encoding='windows-1252'), "html.parser")    
        

        #Getting date4
        date4_element = soup.select_one("span.date")
        date4 = ""
        if date4_element is not None:
            date4 = date4_element.get_text()
        

        # Getting date6

        date6_element = soup.select_one("h1#page-title + p")
        date6 = ""
        if date6_element is not None:
            date6 = date6_element.get_text()



        #deleting all non body text content from the html
        for script in soup(['script', 'style', 'a', 'noscript', 'meta', 'head',  'input', 'form']):
            script.decompose()    # rip it out
            
        # get text
        doctext = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in doctext.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        doctext = '\n'.join(chunk for chunk in chunks if chunk)
        
        
        #old 'simple' cleaning 
        #doctext = soup.get_text().replace("\n", " ")

        with open("{}/{}.txt".format(text_dir, id), "w") as textfile:
            textfile.write(doctext)
        

        #Getting date5

        htmlparser = etree.HTMLParser()    
        try:
            tree = etree.parse(open(path), htmlparser)
        except UnicodeDecodeError:
            tree = etree.parse(open(path, encoding='windows-1252'), htmlparser)
        date5 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 23)")

        # a special cases for date 5
        date5_1 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 25)")


        # Getting date7 from National Archives --> the date the site was archived on --> the date the site was released on would have been even earlier

        date7 = tree.xpath("substring(substring-after(/html/head/script/text(), 'timestamp'),9,14)")


        mytree = html.parse(open(path), htmlparser)

        #Getting date with the htmldate package
        htmldate = find_date(mytree)

        #Getting the main content of the page with the trafilatura package
        main_content = trafilatura.extract(mytree)


        # assinging the already crawled dates
        date1 = line["date1"]
        date2 = line["date2"]


        # Storing the oldest date as final date variable in python date format
        date_vars = [date1, date2, date4, date5, date5_1, date6, date7]
        
        final_date = None 

        for date_var in date_vars:
            try:
                if final_date is None:
                    final_date = parse(date_var, ignoretz = True)
                elif parse(date_var, ignoretz=True) < final_date:
                    final_date = parse(date_var, ignoretz=True)
            except dateutil.parser._parser.ParserError:
                pass 


        #finding the matches for agile and agility
        agile_term = []
        agile_term = agile_regex.findall(doctext.lower())
        

        #finding the matches for agile...methods
        agile_method = []
        agile_method = agile_method_regex.findall(doctext.lower())



        # finding the context for the agile
        agile_context = []
        agile_context = agile_context_regex.search(doctext)

        agile_context_pre = ""
        agile_context_post = ""
        
        if agile_context is not None:
            agile_context_pre = " ".join(agile_context.group(1,2,3,4))
            agile_context_post = " ".join(agile_context.group(5,6,7,8))
        

        # assigning all the variables to the items

        if len(agile_term) == 0:
            line["agile_term"] = "" 
        else:
            line["agile_term"] = ",".join(agile_term)
        
        if len(agile_method) == 0:
            line["agile_method"] = "" 
        else:
            line["agile_method"] = ",".join(agile_method)

        line["agile_context_pre"] = agile_context_pre
        line["agile_context_post"] = agile_context_post 
        line["date4"] = date4
        line["date5"] = date5
        line["date5_1"] = date5_1
        line["date6"] = date6
        line["date7"] = date7
        line["final_date"] = final_date
        line["htmldate"] = htmldate
        line["country"] = "UK"
        line["level"] = state
        line["main_content"] = main_content
        line["text_file_loc"] = "{}/{}.txt".format(text_dir, id)
        
    outputfile = open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/CSVs/UK/{}.csv".format(state), "w")
    writer = csv.DictWriter(outputfile, fieldnames=["id", "country", "level", 'url', 'domain', 'date1', 'date2', 'date3', 'date4', 'date5', 'date5_1', 'date6', 'date7', 'final_date', 'htmldate', 'heading', 'agile_term', 'agile_method', 'agile_context_pre', 'agile_context_post', 'main_content', 'text_file_loc'])
    writer.writeheader()
    for id, line in array.items():
        writer.writerow(line)
    outputfile.close()
    

In [8]:
#Running processing for Germany
process_state_DE("Federal")

processing d64b7c1c-cddf-3ef7-ab10-f6c32100fc36
processing 001213a3-6bb1-374b-83ff-6d166d567933
processing c41d71f9-6ced-3c39-a661-dd85dff2175c
processing a36f0ef0-ec1b-33b6-aad4-45cda5defa48
processing 4d89205c-ee61-3d36-bdcc-d9ec9b22a082
processing 19dd6541-fe8a-3edb-835b-ab6d94423537
processing 447443cc-ba1b-3d2b-b0b7-5390f81e6569
processing af9dd497-d2a2-35ca-b48f-33c26d4476de
processing 9e8cc26a-36ba-3046-bf44-5c1484f2f0b3
processing 42f53033-7a47-39bf-ade6-3e3623083fb2
processing 1c69cbda-86f6-348a-ba31-a53f12a9197e
processing 83d20ea8-c770-37d6-b880-c7f329c5ced5
processing 9f67207b-61f1-3521-88c4-b533f74a244b
processing b0f2757b-c084-34a0-a104-e1bcbae826c8
processing 68d31c56-39f1-3753-b29d-ac5842eac9d3
processing 9e925771-d3a3-3b40-add7-fde6e74d97f2
processing f470ba9c-6aa9-39a0-86e5-d560d928cd16
processing 9b2b3bc8-d57f-345b-a4cc-66acb4144e91
processing 2233b421-720a-391b-b517-ae3a5233afe4
processing 242e8fc5-7383-3e83-bd2c-a00b7ee9b877
processing 635cbcf2-0fe2-39cd-91b5-d1ad7

In [45]:
#Running processing for UK
process_state_UK("General")

processing 74ef9f72-3612-3d2b-a14c-e080bb3567eb


AttributeError: 'lxml.etree._Element' object has no attribute 'text_content'

In [3]:
##### Cleaning State Germany
def cleaning_state_DE(state):
    inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/CSVs/Germany/{}.csv".format(state)
    outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/CSVs/Germany/{}_v1.csv".format(state)

    with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
        df = pd.read_csv(inFile)
        df.sort_values(by=['final_date'])
        df.dropna(axis=0, how='any', thresh=None, subset=(['agile_term', 'agile_context_pre', 'agile_context_post']), inplace=True)
        df.drop_duplicates(subset=(['heading', 'agile_context_pre', 'agile_context_post']), inplace=True)
        df['heading'] = df['heading'].astype(str)
        #deleting false positives
        df = df[~df.agile_context_post.str.startswith("(")] # dropdown menu entry
        df = df[~df.agile_context_pre.str.endswith(")")] # dropdown menu entry

        df.to_csv(outFile)

In [32]:
##### Cleaning State UK
def cleaning_state_UK(state):
    inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/CSVs/UK/{}.csv".format(state)
    outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Processing/DATA/CSVs/UK/{}_v1.csv".format(state)

    with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
        df = pd.read_csv(inFile)
        df.sort_values(by=['final_date'])
        df.dropna(axis=0, how='any', thresh=None, subset=(['agile_term']), inplace=True)
        df.drop_duplicates(subset=(['heading', 'agile_context_pre', 'agile_context_post']), inplace=True)
        df['agile_context_pre'] = df['agile_context_pre'].astype(str)
        df['agile_context_post'] = df['agile_context_post'].astype(str)
        #deleting false positives
        df = df[~df.agile_context_post.str.startswith("(")] # dropdown menu entry
        df = df[~df.agile_context_pre.str.endswith(")")] # dropdown menu entry
        df = df[~df.agile_context_pre.str.endswith("EB")] # supplier list
        df = df[~df.agile_context_post.str.match('and battle-winning armed forces')] #army related agility
        df = df[~df.agile_context_pre.str.match('Atom CategoriesCategories Select Category')] # dopdown menu entry
        df = df[~df.agile_context_pre.str.match('RAF to Force Generate')] # Entry on Air Commanders
        df = df[~df.agile_context_post.str.startswith("Trains")] # name of a company "Agility Trains"
        df = df[~df.agile_context_post.str.startswith("trains")] # name of a company "Agility Trains"
        df = df[~df.agile_context_post.str.match('People Services Limited Independent')] #company name
        df.to_csv(outFile)

In [41]:
#Cleaning State Germany
cleaning_state_DE('insert_state_name')

NameError: name 'cleaning_state_DE' is not defined

In [33]:
# Cleaning State UK
cleaning_state_UK('General')

In [ ]:
################### END OF ANALYSIS ###################

In [ ]:
####### Combining all the Data Frames into one with pandas ########






In [28]:
#tester for getting dates with Xpath

htmlparser = etree.HTMLParser()    
try:
    tree = etree.parse(open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GithubRepository/Analysis/ALL_CRAWLS/Germany/Federal/www.westmidlands-pcc.gov.uk/765598f5-dc1c-3af2-94a6-96bd59ad0c44.html", "r"), htmlparser)
except UnicodeDecodeError:
    tree = etree.parse(open(path, encoding='windows-1252'), htmlparser)
date8 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 25)")
print(date8)



2019-05-29T12:44:02+00:00


In [20]:
#testing regex expressions
agile_regex = re.compile(r'\bagile\b(?:(?![.]|agile).)*?\bmethod\w*?\b', re.IGNORECASE | re.UNICODE)
text = 'agile teams            To help people in policy or related areas understand what digital means in government, exploring the methodology'
print(agile_regex.findall(text))

[' ']


In [4]:
#testing deletion of elements from HTML

markup = '<li class="gem-c-related-navigation__link"><a class="gem-c-related-navigation__section-link gem-c-related-navigation__section-link--sidebar  gem-c-related-navigation__section-link--other" data-track-category="relatedLinkClicked" data-track-action="1.4 Related content" data-track-label="/government/publications/assurance-for-agile-delivery-of-digital-services" href="/government/publications/assurance-for-agile-delivery-of-digital-services">Assurance for agile delivery of digital services</a></li>'
soup = BeautifulSoup(markup)
a_tag = soup.a

soup.i.decompose()

a_tag
# <a href="http://example.com/">I linked to</a>

AttributeError: 'NoneType' object has no attribute 'items'